## # 3D Individual Stock Analysis: Valuation + Risk + Performance
# X-axis: Time, Y-axis: Valuation Percentile, Z-axis: Risk Index

In [ ]:
# 3D Individual Stock Analysis: Valuation + Risk + Performance
# X-axis: Time, Y-axis: Valuation Percentile, Z-axis: Risk Index

# Install required packages
!pip install yfinance fredapi requests pandas numpy matplotlib seaborn plotly scipy kaleido

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import yfinance as yf
from fredapi import Fred
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from scipy.interpolate import griddata
import requests
import warnings
warnings.filterwarnings('ignore')

# API Configuration
FMP_API_KEY = "6JA2MqTGtnHtQIfRMHDmN4Q1nk69smac"
FRED_API_KEY = "9edcfe0388121c02c0de5328c0d75294"

print("Starting 3D Individual Stock Analysis System...")

# =============================================================================
# FMP API FUNCTIONS FOR INDIVIDUAL STOCKS
# =============================================================================

def get_fmp_data(endpoint, params=None):
    """Generic function to fetch data from FMP API"""
    base_url = "https://financialmodelingprep.com/api/v3/"
    url = f"{base_url}{endpoint}"

    default_params = {"apikey": FMP_API_KEY}
    if params:
        default_params.update(params)

    try:
        response = requests.get(url, params=default_params)
        response.raise_for_status()
        data = response.json()
        return data if data else None
    except Exception as e:
        print(f"Error fetching {endpoint}: {e}")
        return None

def get_stock_profile(symbol):
    """Get company profile and basic information"""
    print(f"Fetching company profile for {symbol}...")
    try:
        profile = get_fmp_data(f"profile/{symbol}")

        if profile and isinstance(profile, list) and len(profile) > 0:
            return profile
        elif profile and isinstance(profile, dict):
            return [profile]  # Wrap single dict in list for consistency
        else:
            print(f"No profile data returned for {symbol}, using basic info")
            # Create basic profile from symbol
            return [{
                'symbol': symbol,
                'companyName': f"{symbol} Inc.",
                'sector': "Unknown",
                'industry': "Unknown",
                'exchangeShortName': "Unknown",
                'mktCap': 0
            }]
    except Exception as e:
        print(f"Error fetching profile for {symbol}: {e}")
        return [{
            'symbol': symbol,
            'companyName': f"{symbol} Inc.",
            'sector': "Unknown",
            'industry': "Unknown",
            'exchangeShortName': "Unknown",
            'mktCap': 0
        }]

def get_stock_ratios(symbol, period="quarter", limit=40):
    """Get financial ratios for the stock"""
    print(f"Fetching financial ratios for {symbol}...")
    ratios = get_fmp_data(f"ratios/{symbol}", {"period": period, "limit": limit})
    return pd.DataFrame(ratios) if ratios else None

def get_stock_metrics(symbol, period="quarter", limit=40):
    """Get key metrics for the stock"""
    print(f"Fetching key metrics for {symbol}...")
    metrics = get_fmp_data(f"key-metrics/{symbol}", {"period": period, "limit": limit})
    return pd.DataFrame(metrics) if metrics else None

def get_stock_financials(symbol, period="quarter", limit=40):
    """Get financial statements"""
    print(f"Fetching financial statements for {symbol}...")

    # Income statement
    income = get_fmp_data(f"income-statement/{symbol}", {"period": period, "limit": limit})
    income_df = pd.DataFrame(income) if income else None

    # Balance sheet
    balance = get_fmp_data(f"balance-sheet-statement/{symbol}", {"period": period, "limit": limit})
    balance_df = pd.DataFrame(balance) if balance else None

    # Cash flow
    cashflow = get_fmp_data(f"cash-flow-statement/{symbol}", {"period": period, "limit": limit})
    cashflow_df = pd.DataFrame(cashflow) if cashflow else None

    return income_df, balance_df, cashflow_df

def get_stock_price_history(symbol, start_date='2015-01-01'):
    """Get historical price data"""
    print(f"Fetching price history for {symbol}...")

    try:
        # Try FMP first
        price_data = get_fmp_data(f"historical-price-full/{symbol}",
                                {"from": start_date, "to": datetime.now().strftime('%Y-%m-%d')})

        if price_data and 'historical' in price_data:
            df = pd.DataFrame(price_data['historical'])
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            df.sort_index(inplace=True)
            return df
        else:
            # Fallback to yfinance
            print(f"FMP failed, using yfinance for {symbol}...")
            stock_data = yf.download(symbol, start=start_date, end=datetime.now())
            return stock_data.rename(columns={'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'})

    except Exception as e:
        print(f"Error getting price data: {e}")
        return None

def get_market_data_fred():
    """Get market-wide data from FRED for context"""
    print("Fetching market context data from FRED...")

    try:
        fred = Fred(api_key=FRED_API_KEY)

        # Market context indicators
        market_data = {}

        series_ids = {
            'S&P_500': 'SP500',
            'VIX': 'VIXCLS',
            '10Y_Treasury': 'GS10',
            '2Y_Treasury': 'GS2',
            'Fed_Funds': 'FEDFUNDS',
            'CPI': 'CPIAUCSL',
            'GDP_Growth': 'A191RL1Q225SBEA',
            'Unemployment': 'UNRATE',
            'Dollar_Index': 'DTWEXBGS'
        }

        for name, series_id in series_ids.items():
            try:
                data = fred.get_series(series_id, start='2015-01-01')
                if not data.empty:
                    market_data[name] = data.resample('ME').last()
                    print(f"  {name}: {data.index.max().strftime('%Y-%m-%d')}")
            except Exception as e:
                print(f"  Could not fetch {name}: {str(e)}")

        return pd.DataFrame(market_data) if market_data else None

    except Exception as e:
        print(f"Error fetching FRED data: {e}")
        return None

# =============================================================================
# STOCK-SPECIFIC ANALYSIS FUNCTIONS
# =============================================================================

def calculate_stock_valuation_metrics(ratios_df, metrics_df, price_df):
    """Calculate comprehensive valuation metrics for individual stock"""
    if ratios_df is None or ratios_df.empty:
        return None

    try:
        print("Calculating stock valuation metrics...")

        # Ensure date column is datetime and set as index
        if 'date' in ratios_df.columns:
            ratios_df['date'] = pd.to_datetime(ratios_df['date'])
            ratios_df.set_index('date', inplace=True)

        if metrics_df is not None and 'date' in metrics_df.columns:
            metrics_df['date'] = pd.to_datetime(metrics_df['date'])
            metrics_df.set_index('date', inplace=True)

        valuation_data = {}

        # P/E Ratios
        if 'priceEarningsRatio' in ratios_df.columns:
            valuation_data['PE_Ratio'] = pd.to_numeric(ratios_df['priceEarningsRatio'], errors='coerce')

        # P/B Ratio
        if 'priceToBookRatio' in ratios_df.columns:
            valuation_data['PB_Ratio'] = pd.to_numeric(ratios_df['priceToBookRatio'], errors='coerce')

        # P/S Ratio
        if 'priceToSalesRatio' in ratios_df.columns:
            valuation_data['PS_Ratio'] = pd.to_numeric(ratios_df['priceToSalesRatio'], errors='coerce')

        # EV/EBITDA
        if metrics_df is not None and 'enterpriseValueMultiple' in metrics_df.columns:
            valuation_data['EV_EBITDA'] = pd.to_numeric(metrics_df['enterpriseValueMultiple'], errors='coerce')

        # PEG Ratio
        if 'pegRatio' in ratios_df.columns:
            valuation_data['PEG_Ratio'] = pd.to_numeric(ratios_df['pegRatio'], errors='coerce')

        # Dividend Yield (inverse - higher yield can mean value)
        if 'dividendYield' in ratios_df.columns:
            div_yield = pd.to_numeric(ratios_df['dividendYield'], errors='coerce')
            valuation_data['Div_Yield_Inv'] = -div_yield  # Negative because high yield = lower valuation

        # Price to Cash Flow
        if 'priceCashFlowRatio' in ratios_df.columns:
            valuation_data['PCF_Ratio'] = pd.to_numeric(ratios_df['priceCashFlowRatio'], errors='coerce')

        if not valuation_data:
            print("Warning: No valuation metrics found")
            return None

        # Create DataFrame and calculate z-scores
        val_df = pd.DataFrame(valuation_data)
        val_df.sort_index(inplace=True)

        # Calculate rolling z-scores for each metric (higher z-score = more expensive)
        z_scores = {}
        for col in val_df.columns:
            if val_df[col].notna().sum() > 5:  # Need at least 5 data points
                z_scores[f'{col}_zscore'] = rolling_zscore(val_df[col])

        if z_scores:
            z_df = pd.DataFrame(z_scores)
            # Composite valuation score
            composite_valuation = z_df.mean(axis=1)
            # Convert to percentiles (0-100)
            valuation_percentile = composite_valuation.rank(pct=True) * 100

            return valuation_percentile
        else:
            print("Warning: Could not calculate valuation z-scores")
            return None

    except Exception as e:
        print(f"Error calculating valuation metrics: {e}")
        return None

def calculate_stock_risk_metrics(price_df, ratios_df, market_data):
    """Calculate risk index for individual stock"""
    if price_df is None or price_df.empty:
        return None

    try:
        print("Calculating stock risk metrics...")

        risk_components = {}

        # Price-based risk metrics
        if 'close' in price_df.columns:
            # Returns and volatility
            price_df['returns'] = price_df['close'].pct_change()
            price_df['returns_30d'] = price_df['close'].pct_change(30)
            price_df['volatility_30d'] = price_df['returns'].rolling(30).std() * np.sqrt(252)
            price_df['volatility_90d'] = price_df['returns'].rolling(90).std() * np.sqrt(252)

            # Drawdown
            rolling_max = price_df['close'].rolling(252).max()
            price_df['drawdown'] = (price_df['close'] / rolling_max - 1) * 100

            # Sharp moves (crash indicators)
            price_df['big_decline'] = (price_df['returns'] < -0.05).astype(int) * 3.0
            price_df['severe_drawdown'] = (price_df['drawdown'] < -20).astype(int) * 4.0

            # Monthly aggregation for consistency
            monthly_data = price_df.resample('ME').agg({
                'volatility_30d': 'last',
                'volatility_90d': 'last',
                'drawdown': 'last',
                'big_decline': 'max',
                'severe_drawdown': 'max',
                'returns_30d': 'last'
            })

            risk_components['Volatility_30d'] = monthly_data['volatility_30d'] * 2.0  # Weight
            risk_components['Drawdown'] = -monthly_data['drawdown'] * 0.1  # Convert to positive risk
            risk_components['Big_Declines'] = monthly_data['big_decline']
            risk_components['Severe_Drawdown'] = monthly_data['severe_drawdown']
            risk_components['Negative_Returns'] = -monthly_data['returns_30d'].clip(upper=0) * 10

        # Fundamental risk metrics from ratios
        if ratios_df is not None and not ratios_df.empty:
            # Debt ratios (higher debt = higher risk)
            if 'debtRatio' in ratios_df.columns:
                debt_ratio = pd.to_numeric(ratios_df['debtRatio'], errors='coerce')
                risk_components['Debt_Risk'] = rolling_zscore(debt_ratio) * 1.5

            # Interest coverage (lower coverage = higher risk)
            if 'interestCoverage' in ratios_df.columns:
                int_coverage = pd.to_numeric(ratios_df['interestCoverage'], errors='coerce')
                risk_components['Interest_Risk'] = -rolling_zscore(int_coverage) * 1.5

            # Current ratio (lower = higher liquidity risk)
            if 'currentRatio' in ratios_df.columns:
                current_ratio = pd.to_numeric(ratios_df['currentRatio'], errors='coerce')
                risk_components['Liquidity_Risk'] = -rolling_zscore(current_ratio) * 1.0

        # Market context risk (beta, correlation with market stress)
        if market_data is not None and not market_data.empty:
            # Align dates
            aligned_data = pd.concat([monthly_data, market_data], axis=1, join='inner')

            if len(aligned_data) > 12:  # Need enough data for beta calculation
                # Calculate beta
                if 'returns_30d' in aligned_data.columns and 'S&P_500' in aligned_data.columns:
                    market_returns = aligned_data['S&P_500'].pct_change()
                    stock_returns = aligned_data['returns_30d']

                    # Rolling beta calculation
                    window = 12
                    rolling_beta = []
                    for i in range(window, len(aligned_data)):
                        stock_ret_window = stock_returns.iloc[i-window:i]
                        market_ret_window = market_returns.iloc[i-window:i]

                        if stock_ret_window.notna().sum() > 6 and market_ret_window.notna().sum() > 6:
                            try:
                                beta = np.cov(stock_ret_window.dropna(), market_ret_window.dropna())[0, 1] / np.var(market_ret_window.dropna())
                                rolling_beta.append(beta)
                            except:
                                rolling_beta.append(1.0)
                        else:
                            rolling_beta.append(1.0)

                    beta_series = pd.Series(rolling_beta, index=aligned_data.index[window:])
                    risk_components['Beta_Risk'] = (beta_series - 1).abs() * 1.5  # Distance from market beta

        if not risk_components:
            print("Warning: No risk components calculated")
            return pd.Series(0, index=price_df.resample('ME').last().index)

        # Combine risk components
        risk_df = pd.DataFrame(risk_components)
        risk_df.fillna(0, inplace=True)

        # Calculate composite risk score
        composite_risk = risk_df.mean(axis=1)

        # Apply rolling z-score normalization
        risk_index = rolling_zscore(composite_risk, window=12, min_periods=6)

        return risk_index

    except Exception as e:
        print(f"Error calculating risk metrics: {e}")
        return None

def rolling_zscore(series, window=12, min_periods=6):
    """Calculate rolling z-score for time series"""
    if series is None or series.empty:
        return pd.Series(dtype=float)

    # Convert to numeric, handling any string values
    series = pd.to_numeric(series, errors='coerce')

    rolling_mean = series.rolling(window=window, min_periods=min_periods).mean()
    rolling_std = series.rolling(window=window, min_periods=min_periods).std()

    # Avoid division by zero
    rolling_std = rolling_std.replace(0, np.nan)

    zscore = (series - rolling_mean) / rolling_std

    # Handle extreme values
    zscore = np.sign(zscore) * np.minimum(np.abs(zscore), 3)  # Cap at +/- 3

    return zscore.fillna(0)

def create_stock_3d_visualization(symbol, valuation_percentile, risk_index, price_df, company_profile=None):
    """Create 3D visualization for individual stock"""
    if valuation_percentile is None or risk_index is None or price_df is None:
        print("Insufficient data for 3D visualization")
        return

    try:
        print(f"Creating 3D visualization for {symbol}...")

        # Align all data by date
        combined_data = pd.DataFrame({
            'valuation_percentile': valuation_percentile,
            'risk_index': risk_index
        })

        # Get price data at monthly frequency
        monthly_prices = price_df.resample('ME').last()['close'] if 'close' in price_df.columns else price_df.resample('ME').last().iloc[:, 0]

        # Align dates
        combined_data = combined_data.join(monthly_prices.rename('price'), how='inner')
        combined_data.dropna(inplace=True)

        if len(combined_data) < 5:
            print("Insufficient aligned data for visualization")
            return

        # Prepare 3D coordinates
        dates = combined_data.index
        dates_numeric = np.array([(date - dates.min()).days for date in dates])  # X-axis
        valuation_y = combined_data['valuation_percentile'].values  # Y-axis
        risk_z = combined_data['risk_index'].values  # Z-axis
        prices = combined_data['price'].values

        # Color by performance and danger level
        price_performance = (prices / prices[0] - 1) * 100  # % change from start
        danger_level = (valuation_y / 100 * 0.5) + ((risk_z + 3) / 6 * 0.5)  # Normalized danger

        # Size by price level
        price_sizes = (prices / prices.max()) * 10 + 3

        # Create main 3D plot
        fig = go.Figure()

        # Main trajectory
        fig.add_trace(go.Scatter3d(
            x=dates_numeric,
            y=valuation_y,
            z=risk_z,
            mode='markers+lines',
            marker=dict(
                size=price_sizes,
                color=danger_level,
                colorscale='RdYlGn_r',
                opacity=0.8,
                colorbar=dict(
                    title=f"{symbol} Risk Level<br>(Red=High Risk+Expensive<br>Green=Low Risk+Cheap)",
                    titleside="right",
                    x=1.02
                ),
                line=dict(color='rgba(0,0,0,0.2)', width=1)
            ),
            line=dict(
                color='rgba(50,50,50,0.5)',
                width=3
            ),
            text=[f"Date: {date.strftime('%Y-%m')}<br>" +
                  f"Price: ${price:.2f}<br>" +
                  f"Valuation: {val:.1f}%<br>" +
                  f"Risk: {risk:.2f}<br>" +
                  f"Performance: {perf:+.1f}%"
                  for date, price, val, risk, perf in
                  zip(dates, prices, valuation_y, risk_z, price_performance)],
            hovertemplate='%{text}<extra></extra>',
            name=f'{symbol} Evolution'
        ))

        # Current position
        if len(dates) > 0:
            fig.add_trace(go.Scatter3d(
                x=[dates_numeric[-1]],
                y=[valuation_y[-1]],
                z=[risk_z[-1]],
                mode='markers',
                marker=dict(
                    size=20,
                    color='gold',
                    symbol='diamond',
                    opacity=1.0,
                    line=dict(color='black', width=3)
                ),
                text=f"CURRENT: {symbol}<br>{dates[-1].strftime('%Y-%m')}<br>Price: ${prices[-1]:.2f}<br>Valuation: {valuation_y[-1]:.1f}%<br>Risk: {risk_z[-1]:.2f}",
                hovertemplate='%{text}<extra></extra>',
                name='Current Position'
            ))

        # Add reference planes
        # High risk plane
        x_plane = [dates_numeric.min(), dates_numeric.max(), dates_numeric.max(), dates_numeric.min()]
        y_plane = [0, 0, 100, 100]
        z_plane = [2.0, 2.0, 2.0, 2.0]

        fig.add_trace(go.Mesh3d(
            x=x_plane, y=y_plane, z=z_plane,
            opacity=0.1, color='red',
            name='High Risk Zone', hoverinfo='skip'
        ))

        # High valuation plane
        x_plane = [dates_numeric.min(), dates_numeric.max(), dates_numeric.max(), dates_numeric.min()]
        y_plane = [80, 80, 80, 80]
        z_plane = [-3, -3, 4, 4]

        fig.add_trace(go.Mesh3d(
            x=x_plane, y=y_plane, z=z_plane,
            opacity=0.1, color='orange',
            name='High Valuation Zone', hoverinfo='skip'
        ))

        # Update layout
        company_name = company_profile[0]['companyName'] if company_profile else symbol
        sector = company_profile[0]['sector'] if company_profile else "Unknown"

        fig.update_layout(
            title={
                'text': f'3D Analysis: {company_name} ({symbol})<br><sub>Sector: {sector} | Time × Valuation × Risk</sub>',
                'x': 0.5,
                'font': {'size': 16}
            },
            scene=dict(
                xaxis=dict(
                    title='Time (Days)',
                    tickvals=dates_numeric[::max(1, len(dates_numeric)//8)],
                    ticktext=[dates[i].strftime('%Y-%m') for i in range(0, len(dates), max(1, len(dates)//8))]
                ),
                yaxis=dict(title='Valuation Percentile (%)', range=[0, 100]),
                zaxis=dict(title='Risk Index'),
                bgcolor='rgba(240,240,240,0.9)',
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
            ),
            width=1200,
            height=800,
            margin=dict(l=0, r=100, b=0, t=100)
        )

        fig.show()

        return fig

    except Exception as e:
        print(f"Error creating 3D visualization: {e}")
        import traceback
        traceback.print_exc()

def create_stock_dashboard(symbol, price_df, valuation_percentile, risk_index, ratios_df, company_profile):
    """Create comprehensive dashboard for the stock"""
    try:
        print(f"Creating dashboard for {symbol}...")

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                f'{symbol} Price & Risk Over Time',
                'Valuation vs Risk Relationship',
                'Key Financial Ratios',
                'Performance Metrics'
            ),
            specs=[[{"secondary_y": True}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": True}]],
            vertical_spacing=0.12
        )

        # Plot 1: Price and Risk over time
        monthly_prices = price_df.resample('ME').last()['close'] if 'close' in price_df.columns else price_df.resample('ME').last().iloc[:, 0]

        fig.add_trace(
            go.Scatter(x=monthly_prices.index, y=monthly_prices.values,
                      name='Stock Price', line=dict(color='blue', width=2)),
            row=1, col=1
        )

        if risk_index is not None:
            fig.add_trace(
                go.Scatter(x=risk_index.index, y=risk_index.values,
                          name='Risk Index', line=dict(color='red', width=2)),
                row=1, col=1, secondary_y=True
            )

        # Plot 2: Valuation vs Risk scatter
        if valuation_percentile is not None and risk_index is not None:
            # Align data
            aligned = pd.DataFrame({'val': valuation_percentile, 'risk': risk_index}).dropna()

            if len(aligned) > 0:
                time_colors = [(date - aligned.index.min()).days for date in aligned.index]

                fig.add_trace(
                    go.Scatter(x=aligned['val'], y=aligned['risk'],
                              mode='markers',
                              marker=dict(
                                  size=8, color=time_colors, colorscale='Viridis',
                                  colorbar=dict(title="Time", x=0.48, len=0.4)
                              ),
                              name='Evolution',
                              text=[date.strftime('%Y-%m') for date in aligned.index]),
                    row=1, col=2
                )

                # Current position
                fig.add_trace(
                    go.Scatter(x=[aligned['val'].iloc[-1]], y=[aligned['risk'].iloc[-1]],
                              mode='markers', marker=dict(size=15, color='gold', symbol='star'),
                              name='Current'),
                    row=1, col=2
                )

        # Plot 3: Key ratios over time
        if ratios_df is not None and not ratios_df.empty:
            key_ratios = ['priceEarningsRatio', 'priceToBookRatio', 'debtRatio']
            colors = ['green', 'blue', 'red']

            for i, ratio in enumerate(key_ratios):
                if ratio in ratios_df.columns:
                    values = pd.to_numeric(ratios_df[ratio], errors='coerce').dropna()
                    if len(values) > 0:
                        fig.add_trace(
                            go.Scatter(x=values.index, y=values.values,
                                      name=ratio.replace('Ratio', '').replace('price', 'P/'),
                                      line=dict(color=colors[i % len(colors)])),
                            row=2, col=1
                        )

        # Plot 4: Performance comparison
        if len(monthly_prices) > 1:
            returns = monthly_prices.pct_change().fillna(0)
            cumulative_returns = (1 + returns).cumprod() - 1

            fig.add_trace(
                go.Scatter(x=cumulative_returns.index, y=cumulative_returns.values * 100,
                          name='Cumulative Return (%)', line=dict(color='purple', width=2)),
                row=2, col=2
            )

            # Rolling volatility
            rolling_vol = returns.rolling(12).std() * np.sqrt(12) * 100
            fig.add_trace(
                go.Scatter(x=rolling_vol.index, y=rolling_vol.values,
                          name='12M Volatility (%)', line=dict(color='orange', width=1)),
                row=2, col=2, secondary_y=True
            )

        # Update layout
        company_name = company_profile[0]['companyName'] if company_profile else symbol
        fig.update_layout(
            title_text=f"{company_name} ({symbol}) - Comprehensive Analysis Dashboard",
            height=800, showlegend=True
        )

        fig.show()

    except Exception as e:
        print(f"Error creating dashboard: {e}")

def analyze_stock_position(symbol, valuation_percentile, risk_index, company_profile):
    """Provide comprehensive analysis of current stock position"""
    if valuation_percentile is None or risk_index is None:
        print("Insufficient data for analysis")
        return

    try:
        current_val = valuation_percentile.iloc[-1]
        current_risk = risk_index.iloc[-1]
        latest_date = valuation_percentile.index[-1]

        print("\n" + "="*80)
        print(f"3D STOCK ANALYSIS: {symbol}")
        print("="*80)

        if company_profile:
            print(f"Company: {company_profile[0]['companyName']}")
            print(f"Sector: {company_profile[0]['sector']}")
            print(f"Industry: {company_profile[0]['industry']}")
            print(f"Market Cap: ${company_profile[0]['mktCap']:,}" if 'mktCap' in company_profile[0] else "")

        print(f"Analysis Date: {latest_date.strftime('%Y-%m-%d')}")
        print("-"*80)

        # Current coordinates
        print("\n1. CURRENT 3D COORDINATES:")
        print(f"   Y-axis (Valuation): {current_val:.1f}%")
        print(f"   Z-axis (Risk): {current_risk:.2f}")

        # Quadrant analysis
        print("\n2. INVESTMENT QUADRANT:")
        high_val = current_val > 70
        high_risk = current_risk > 1.0

        if high_val and high_risk:
            quadrant = "AVOID ZONE"
            description = "Expensive + Risky = High probability of poor returns"
            action = "SELL or AVOID - Wait for better entry"
            color = "RED ZONE"
        elif high_val and not high_risk:
            quadrant = "EXPENSIVE BUT STABLE"
            description = "High valuation but low risk = Premium priced quality"
            action = "HOLD if owned, be selective on new positions"
            color = "YELLOW ZONE"
        elif not high_val and high_risk:
            quadrant = "OPPORTUNITY ZONE"
            description = "Reasonable valuation + High risk = Potential value play"
            action = "RESEARCH further - potential contrarian opportunity"
            color = "ORANGE ZONE"
        else:
            quadrant = "BUY ZONE"
            description = "Reasonable valuation + Low risk = Attractive opportunity"
            action = "STRONG BUY - favorable risk/reward"
            color = "GREEN ZONE"

        print(f"   Quadrant: {quadrant}")
        print(f"   Description: {description}")
        print(f"   Recommendation: {action}")
        print(f"   3D Zone: {color}")

        # Historical context
        val_percentile_hist = stats.percentileofscore(valuation_percentile.dropna(), current_val)
        risk_percentile_hist = stats.percentileofscore(risk_index.dropna(), current_risk)

        print("\n3. HISTORICAL CONTEXT:")
        print(f"   Valuation vs Own History: {val_percentile_hist:.1f}%")
        print(f"   Risk vs Own History: {risk_percentile_hist:.1f}%")

        # Trend analysis
        val_1m = valuation_percentile.diff().iloc[-1] if len(valuation_percentile) > 1 else 0
        val_3m = valuation_percentile.diff(3).iloc[-1] if len(valuation_percentile) > 3 else 0
        risk_1m = risk_index.diff().iloc[-1] if len(risk_index) > 1 else 0
        risk_3m = risk_index.diff(3).iloc[-1] if len(risk_index) > 3 else 0

        print("\n4. TRAJECTORY ANALYSIS:")
        print(f"   Valuation Trend (1M): {val_1m:+.1f}")
        print(f"   Valuation Trend (3M): {val_3m:+.1f}")
        print(f"   Risk Trend (1M): {risk_1m:+.2f}")
        print(f"   Risk Trend (3M): {risk_3m:+.2f}")

        # Investment sizing recommendation
        print("\n5. POSITION SIZING GUIDE:")
        if current_risk > 2.0:
            sizing = "MICRO (0-2% of portfolio)"
        elif current_risk > 1.0:
            sizing = "SMALL (2-5% of portfolio)"
        elif current_risk > 0:
            sizing = "NORMAL (5-10% of portfolio)"
        else:
            sizing = "LARGE (10-15% of portfolio)"

        print(f"   Recommended Position Size: {sizing}")

        # Risk management
        print("\n6. RISK MANAGEMENT:")
        if current_risk > 1.5:
            print("   Stop Loss: 10-15% below current price")
            print("   Monitor: Weekly basis")
        elif current_risk > 0.5:
            print("   Stop Loss: 15-20% below current price")
            print("   Monitor: Bi-weekly basis")
        else:
            print("   Stop Loss: 20-25% below current price")
            print("   Monitor: Monthly basis")

        print("\n" + "="*80)

    except Exception as e:
        print(f"Error in stock analysis: {e}")

# =============================================================================
# MAIN EXECUTION FUNCTION FOR INDIVIDUAL STOCKS
# =============================================================================

def analyze_stock(symbol, start_date='2015-01-01'):
    """Main function to perform comprehensive 3D analysis of individual stock"""
    try:
        print(f"Starting comprehensive 3D analysis for {symbol.upper()}")
        print("="*60)

        # Get company profile with better error handling
        company_profile = get_stock_profile(symbol.upper())

        if company_profile and len(company_profile) > 0:
            profile = company_profile[0]
            print(f"Company: {profile.get('companyName', f'{symbol} Inc.')}")
            print(f"Sector: {profile.get('sector', 'Unknown')}")
            print(f"Exchange: {profile.get('exchangeShortName', 'Unknown')}")
            if 'mktCap' in profile and profile['mktCap']:
                print(f"Market Cap: ${profile['mktCap']:,}")
        else:
            print(f"Using basic profile for {symbol}")

        # Get financial data with better error handling
        print("\nFetching financial data...")
        ratios_df = get_stock_ratios(symbol.upper())
        metrics_df = get_stock_metrics(symbol.upper())

        # Try to get financial statements (optional)
        try:
            income_df, balance_df, cashflow_df = get_stock_financials(symbol.upper())
        except:
            income_df = balance_df = cashflow_df = None
            print("  Financial statements not available")

        # Get price history (essential)
        print("\nFetching price data...")
        price_df = get_stock_price_history(symbol.upper(), start_date)

        if price_df is None or price_df.empty:
            print(f"ERROR: Could not retrieve price data for {symbol}")
            print("This is essential for analysis. Please check:")
            print("1. Symbol spelling is correct")
            print("2. Symbol exists and is publicly traded")
            print("3. API connectivity")
            return None

        print(f"✓ Price data: {len(price_df)} records from {price_df.index.min().strftime('%Y-%m-%d')} to {price_df.index.max().strftime('%Y-%m-%d')}")

        # Get market context (optional but helpful)
        print("\nFetching market context...")
        market_data = get_market_data_fred()
        if market_data is not None:
            print(f"✓ Market data: {len(market_data)} records")
        else:
            print("  Market context not available - will use stock-only analysis")

        # Calculate metrics
        print("\nCalculating analysis metrics...")

        # Calculate valuation percentiles (Y-axis)
        print("  Computing valuation metrics...")
        valuation_percentile = calculate_stock_valuation_metrics(ratios_df, metrics_df, price_df)

        if valuation_percentile is not None:
            print(f"  ✓ Valuation percentile calculated ({len(valuation_percentile)} points)")
        else:
            print("  ⚠ Using neutral valuation (limited fundamental data)")
            monthly_index = price_df.resample('ME').last().index
            valuation_percentile = pd.Series(50, index=monthly_index, name='valuation_percentile')

        # Calculate risk index (Z-axis)
        print("  Computing risk metrics...")
        risk_index = calculate_stock_risk_metrics(price_df, ratios_df, market_data)

        if risk_index is not None:
            print(f"  ✓ Risk index calculated ({len(risk_index)} points)")
        else:
            print("  ⚠ Using neutral risk index")
            monthly_index = price_df.resample('ME').last().index
            risk_index = pd.Series(0, index=monthly_index, name='risk_index')

        # Ensure we have sufficient data
        if len(valuation_percentile) < 3 or len(risk_index) < 3:
            print("ERROR: Insufficient data points for meaningful analysis")
            print(f"Valuation points: {len(valuation_percentile)}, Risk points: {len(risk_index)}")
            return None

        print("\nCreating visualizations...")

        # Create 3D visualization
        try:
            fig = create_stock_3d_visualization(symbol.upper(), valuation_percentile, risk_index, price_df, company_profile)
            if fig:
                print("✓ 3D visualization created")
        except Exception as e:
            print(f"⚠ Could not create 3D visualization: {e}")

        # Create dashboard
        try:
            create_stock_dashboard(symbol.upper(), price_df, valuation_percentile, risk_index, ratios_df, company_profile)
            print("✓ Dashboard created")
        except Exception as e:
            print(f"⚠ Could not create dashboard: {e}")

        # Print analysis
        try:
            analyze_stock_position(symbol.upper(), valuation_percentile, risk_index, company_profile)
        except Exception as e:
            print(f"⚠ Could not complete position analysis: {e}")

        # Save results
        print(f"\nSaving results for {symbol.upper()}...")
        try:
            if valuation_percentile is not None and len(valuation_percentile) > 0:
                valuation_percentile.to_csv(f'{symbol.upper()}_valuation_percentile.csv')
                print(f"✓ {symbol.upper()}_valuation_percentile.csv")

            if risk_index is not None and len(risk_index) > 0:
                risk_index.to_csv(f'{symbol.upper()}_risk_index.csv')
                print(f"✓ {symbol.upper()}_risk_index.csv")

            price_df.to_csv(f'{symbol.upper()}_price_data.csv')
            print(f"✓ {symbol.upper()}_price_data.csv")

        except Exception as e:
            print(f"⚠ Error saving files: {e}")

        # Return results
        results = {
            'symbol': symbol.upper(),
            'analysis_date': datetime.now().strftime('%Y-%m-%d'),
            'valuation_percentile': valuation_percentile,
            'risk_index': risk_index,
            'price_data': price_df,
            'company_profile': company_profile
        }

        print(f"\n✓ Analysis complete for {symbol.upper()}!")
        return results

    except Exception as e:
        print(f"ERROR analyzing {symbol}: {str(e)}")
        import traceback
        print("Full traceback:")
        traceback.print_exc()
        return None

def compare_stocks(symbols, start_date='2015-01-01'):
    """Compare multiple stocks in 3D space"""
    try:
        print(f"Comparing multiple stocks: {', '.join(symbols)}")
        print("="*60)

        stock_data = {}

        # Analyze each stock
        for symbol in symbols:
            print(f"\nAnalyzing {symbol}...")
            result = analyze_stock(symbol, start_date)
            if result:
                stock_data[symbol.upper()] = result

        if len(stock_data) < 2:
            print("Need at least 2 stocks for comparison")
            return

        # Create comparison 3D plot
        fig = go.Figure()

        colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']

        for i, (symbol, data) in enumerate(stock_data.items()):
            valuation = data['valuation_percentile']
            risk = data['risk_index']

            if valuation is not None and risk is not None:
                # Align data
                aligned = pd.DataFrame({'val': valuation, 'risk': risk}).dropna()

                if len(aligned) > 0:
                    dates_numeric = [(date - aligned.index.min()).days for date in aligned.index]

                    fig.add_trace(go.Scatter3d(
                        x=dates_numeric,
                        y=aligned['val'].values,
                        z=aligned['risk'].values,
                        mode='markers+lines',
                        marker=dict(size=6, color=colors[i % len(colors)], opacity=0.7),
                        line=dict(color=colors[i % len(colors)], width=2),
                        name=symbol,
                        text=[f"{symbol}<br>{date.strftime('%Y-%m')}<br>Val: {val:.1f}%<br>Risk: {risk:.2f}"
                              for date, val, risk in zip(aligned.index, aligned['val'], aligned['risk'])],
                        hovertemplate='%{text}<extra></extra>'
                    ))

                    # Current position
                    fig.add_trace(go.Scatter3d(
                        x=[dates_numeric[-1]],
                        y=[aligned['val'].iloc[-1]],
                        z=[aligned['risk'].iloc[-1]],
                        mode='markers',
                        marker=dict(size=12, color=colors[i % len(colors)], symbol='diamond', opacity=1.0),
                        name=f'{symbol} Current',
                        showlegend=False,
                        hovertemplate=f'{symbol} Current<extra></extra>'
                    ))

        fig.update_layout(
            title='Multi-Stock 3D Comparison: Time × Valuation × Risk',
            scene=dict(
                xaxis_title='Time (Days)',
                yaxis_title='Valuation Percentile (%)',
                zaxis_title='Risk Index',
                bgcolor='rgba(240,240,240,0.9)'
            ),
            width=1200,
            height=800
        )

        fig.show()

        print("Multi-stock comparison complete!")

    except Exception as e:
        print(f"Error in stock comparison: {e}")

# =============================================================================
# EXAMPLE USAGE AND INSTRUCTIONS
# =============================================================================

def main():
    """Interactive main function"""
    print("3D INDIVIDUAL STOCK ANALYSIS SYSTEM")
    print("="*50)
    print("Examples:")
    print("  Single stock: analyze_stock('AAPL')")
    print("  Multiple stocks: compare_stocks(['AAPL', 'GOOGL', 'MSFT'])")
    print("  Custom date: analyze_stock('TSLA', '2020-01-01')")
    print("\nPopular stocks to try: AAPL, GOOGL, MSFT, TSLA, AMZN, META, NVDA")

    # Example analysis
    example_stock = input("\nEnter a stock symbol to analyze (or press Enter for AAPL): ").strip().upper()
    if not example_stock:
        example_stock = "AAPL"

    analyze_stock(example_stock)

if __name__ == "__main__":
    main()

# Usage Instructions
print("""
3D INDIVIDUAL STOCK ANALYSIS INSTRUCTIONS:

BASIC USAGE:
  analyze_stock('AAPL')                    # Analyze Apple
  analyze_stock('TSLA', '2020-01-01')     # Tesla from 2020
  compare_stocks(['AAPL', 'GOOGL'])       # Compare multiple stocks

WHAT YOU GET:
  • 3D plot: Time (X) × Valuation % (Y) × Risk Index (Z)
  • Interactive dashboard with price, ratios, performance
  • Investment quadrant analysis and recommendations
  • Historical context and trend analysis
  • Position sizing and risk management guidance

KEY METRICS USED:
  Valuation (Y-axis):
    • P/E, P/B, P/S ratios
    • EV/EBITDA, PEG ratio
    • Dividend yield, Price/Cash Flow

  Risk (Z-axis):
    • Price volatility and drawdowns
    • Debt ratios and interest coverage
    • Beta vs market
    • Sharp decline indicators

INTERPRETATION:
  • HIGH Y + HIGH Z = AVOID (expensive + risky)
  • HIGH Y + LOW Z = HOLD (expensive but stable)
  • LOW Y + HIGH Z = RESEARCH (cheap but risky)
  • LOW Y + LOW Z = BUY (cheap + safe)

FILES CREATED:
  • {SYMBOL}_valuation_percentile.csv
  • {SYMBOL}_risk_index.csv
  • {SYMBOL}_price_data.csv

STOCK SYMBOLS:
  Use any valid ticker: AAPL, GOOGL, MSFT, TSLA, AMZN, etc.
""")

Starting 3D Individual Stock Analysis System...
3D INDIVIDUAL STOCK ANALYSIS SYSTEM
Examples:
  Single stock: analyze_stock('AAPL')
  Multiple stocks: compare_stocks(['AAPL', 'GOOGL', 'MSFT'])
  Custom date: analyze_stock('TSLA', '2020-01-01')

Popular stocks to try: AAPL, GOOGL, MSFT, TSLA, AMZN, META, NVDA

Enter a stock symbol to analyze (or press Enter for AAPL): TSLA
Starting comprehensive 3D analysis for TSLA
Fetching company profile for TSLA...
Company: Tesla, Inc.
Sector: Consumer Cyclical
Exchange: NASDAQ
Market Cap: $1,127,617,320,000

Fetching financial data...
Fetching financial ratios for TSLA...
Fetching key metrics for TSLA...
Fetching financial statements for TSLA...

Fetching price data...
Fetching price history for TSLA...
✓ Price data: 2679 records from 2015-01-02 to 2025-08-27

Fetching market context...
Fetching market context data from FRED...
  S&P_500: 2025-08-27
  VIX: 2025-08-26
  10Y_Treasury: 2025-07-01
  2Y_Treasury: 2025-07-01
  Fed_Funds: 2025-07-01
  CPI

✓ 3D visualization created
Creating dashboard for TSLA...


✓ Dashboard created

3D STOCK ANALYSIS: TSLA
Company: Tesla, Inc.
Sector: Consumer Cyclical
Industry: Auto - Manufacturers
Market Cap: $1,127,617,320,000
Analysis Date: 2025-06-30
--------------------------------------------------------------------------------

1. CURRENT 3D COORDINATES:
   Y-axis (Valuation): 70.0%
   Z-axis (Risk): -0.10

2. INVESTMENT QUADRANT:
   Quadrant: BUY ZONE
   Description: Reasonable valuation + Low risk = Attractive opportunity
   Recommendation: STRONG BUY - favorable risk/reward
   3D Zone: GREEN ZONE

3. HISTORICAL CONTEXT:
   Valuation vs Own History: 70.0%
   Risk vs Own History: 33.6%

4. TRAJECTORY ANALYSIS:
   Valuation Trend (1M): -20.0
   Valuation Trend (3M): +27.5
   Risk Trend (1M): -0.95
   Risk Trend (3M): -0.48

5. POSITION SIZING GUIDE:
   Recommended Position Size: LARGE (10-15% of portfolio)

6. RISK MANAGEMENT:
   Stop Loss: 20-25% below current price
   Monitor: Monthly basis


Saving results for TSLA...
✓ TSLA_valuation_percentile.csv